Mislio sam koristiti PyTorch jer sigurno neću ručno sve pisati ako se budem ikad ovim bavio, ali sam odlučio iskoristiti "biblioteku" sa laboratorijskih da se bolje upoznam sa stvanom funkcionalnošću takvih metoda.

In [3]:
import numpy as np
from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, accuracy_score

In [4]:
class NeuralNetwork:
    def __init__(self, layers):
        self.layers = layers

    # sekvencijalna propagacija x za svaki sloj 
    # rezultat svakog postaje ulaz za sljedeci sloj
    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x

    # uzima gradijent grad i sekvencijalno primjenjuje operaciju propagacije unazad za svaki sloj (koji su u obrnutom redoslijedu)
    # gradijent se azurira i prosljedjuje prethodnom sloju kao ulaz
    def backward(self, grad):
        for layer in reversed(self.layers):
            grad = layer.backward(grad)
        return grad

    # vraca listu svih parametara 
    def parameters(self):
        params = []
        for layer in self.layers:
            if hasattr(layer, 'parameters') and callable(getattr(layer, 'parameters')):
                params.extend(layer.parameters())
        return params

    # resetuje gradijente svih slojeva
    def zero_grad(self):
        for layer in self.layers:
            if hasattr(layer, 'zero_grad') and callable(getattr(layer, 'zero_grad')):
                layer.zero_grad()

In [5]:
class Linear:
    # nasumicno inicijalizuje tezinsku matricu u odredjenom opsegu, kao i matricu pomjeraja u manjem opsegu
    # inicijalizuje gradijente kao nulte nizove istog oblika kao respektivne matrice
    def __init__(self, in_features, out_features):
        limit = np.sqrt(6 / (in_features + out_features))
        self.W = np.random.uniform(-limit, limit, (out_features, in_features))
        limit /= 100
        self.b = np.random.uniform(-limit, limit, out_features)

        self.grad_w = np.zeros_like(self.W)
        self.grad_b = np.zeros_like(self.b)
    
    # prolaz kroz linearni sloj
    # mnozi numpy niz x sa transponovanom tezinskom matricom i dodaje pomjeraj b sto se vraca kao izlaz sloja 
    def forward(self, x: np.ndarray):
        self.x = x
        return np.matmul(x, self.W.T) + self.b

    # propagacija unazad kroz sloj
    # racuna gradijente sa parametrima sloja i azurira ih sumirajuci ih
    # vraca gradijent mnozeci proslijedjeni (grad) sa tezinskom matricom
    def backward(self, grad: np.ndarray):
        self.grad_w += np.matmul(grad.T, self.x)
        self.grad_b += np.sum(grad, axis=0)
        return np.matmul(grad, self.W)

    # vraca parametre, koristiti se u NeuralNetwork klasi
    def parameters(self):
        return [[self.W, self.grad_w], [self.b, self.grad_b]]

    # resetuje gradijente sloja
    def zero_grad(self):
        self.grad_w.fill(0)
        self.grad_b.fill(0)

In [6]:
class Sigmoid:
    # prolaz sigmoid aktivacione funkcije
    # uzima numpy niz i primjenjuje sigmoid funkciju
    # to cuva u self.y, ali je i vraca
    def forward(self, x):
        self.y = np.divide(1, 1 + np.exp(-x))
        return self.y

    # propagacija unalaz sigmoid aktivacione funkcije
    # uzima numpy niz grad i vraca ga pomnozen sa izvodom sigmoid funkcije
    def backward(self, grad):
        return grad * self.y * (1 - self.y)

In [7]:
# normalizuje izlaz tako da predstavljaju vjerovatnoce cija suma mora biti 1
class Softmax:
    # prolaz kroz softmax aktivacionu funkciju 
    # primjenjuje softmax funkciju na ulaz da bi izlaz predstavljao vjerovatnoce
    def forward(self, x):
        # racuna se maksimalna vrijednost kroz zadnju osu x-a i oduzima se od od x (opet numpy niz) da se osigura numericka stabilnost onemogucujuci velike eksponencijalne vrijednosti
        e_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
        # rezultat se vraca kao izlaz aktivacione funkcije
        return e_x / np.sum(e_x, axis=-1, keepdims=True)

    # propagacija unazad kroz softmax
    # posto softmax ne uvodi nikakvu nelinearnost gradijent ostaje nepromijenjen
    def backward(self, grad):
        return grad

In [8]:
class CrossEntropyLoss:
    def forward(self, y_pred, y):
        self.y_pred = y_pred
        self.y = y
        # + 1e-7 da se ne racuna logaritam nule
        return -np.sum(y * np.log(y_pred + 1e-7)) / len(y)

    # racuna gradijent greske
    def backward(self):
        return (self.y_pred - self.y) / len(self.y)

In [9]:
class SGD:
    def __init__(self, parameters, lr):
        # lista nizova parametara sa gradijentima
        self.parameters = parameters
        # learning rate
        self.lr = lr

    # azuriranje parametara
    def step(self):
        for param, grad in self.parameters:
            param -= self.lr * grad

In [13]:
def evaluate_model(model, x, y):
    y_pred = model.forward(x)
    y_pred_labels = np.argmax(y_pred, axis=1) + 1
    y_true_labels = np.argmax(y, axis=1) + 1
    precision = precision_score(y_true_labels, y_pred_labels, average='weighted', zero_division=0)
    recall = recall_score(y_true_labels, y_pred_labels, average='weighted', zero_division=0)
    # print(f"Preciznost: {precision:.5f}")
    # print(f"Odziv: {recall:.5f}")
    return precision, recall

In [28]:
# hiperparametri
epochs = 5
#learning_rate = 0.35
#batch_size = 12

# input/output velicine
in_features = 54    # 54 je dimenzija podataka, tako i na sajtu pise, pa cu tako koristiti za unos 
out_features = 7    # 7 klasa

# koliko se epoha tolerise (2. dio zadatka)
tolerancy = 2

# prostor hiperparametara (3. dio zadatka)
parameter_grid = {
    'learning_rate': [0.35, 0.035],
    'hidden_units': [(100, 50), (200, 100)],
    'batch_size': [12, 24]
}
# sve kombinacije parametara
parameter_combinations = list(ParameterGrid(parameter_grid))  # htio sam staviti po 3 za svaku, ali to je 27 kombinacija pa mi je predugo za isprobavanje
best_precision = 0.0
best_parameters = None

# ucitavanje, preprocesiranje, oblikovanje i normalizacija unosa
covtype_dataset = fetch_covtype()

x_train, y_train = covtype_dataset.data, covtype_dataset.target
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = (x_train - np.mean(x_train)) / np.std(x_train)
y_train = np.eye(out_features)[y_train - 1]

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42) # 20% je valjda dovoljno za validaciju?

x_test, y_test = covtype_dataset.data, covtype_dataset.target
x_test, y_test = np.array(x_test), np.array(y_test)
x_test = (x_test - np.mean(x_test)) / np.std(x_test)
y_test = np.eye(out_features)[y_test - 1]

In [29]:
for parameters in parameter_combinations:
    # model mreze
    model = NeuralNetwork(
        [
            Linear(in_features, parameters['hidden_units'][0]),
            Sigmoid(),
            Linear(parameters['hidden_units'][0], parameters['hidden_units'][1]),
            Sigmoid(),
            Linear(parameters['hidden_units'][1], out_features),
            Softmax()
        ])

    # funkcija greske, racuna nesaglasnost predvidjenog i stvarnog
    loss_fn = CrossEntropyLoss()
    # stohasticki gradijentni spust, namjesta parametre da se ta greska smanji
    optimizer = SGD(model.parameters(), parameters['learning_rate'])

    print("Parameters: ", parameters)
    eval_prec, eval_rec = evaluate_model(model, x_test, y_test)
    print(f"Precision before training: {eval_prec:.5f}")
    print(f"Recall before training: {eval_rec:.5f}")

    batch_size = parameters['batch_size'] # pravi probleme u f-stringu ako ne stavim u varijablu

    lowest_loss = float('inf')
    num_bad_epochs = 0

    # treniranje
    for epoch in range(epochs):
        total_misclassified = 0
        # mijesanje da se izbjegne overfittovanje specificnom redoslijedu
        permutation = np.random.permutation(x_train.shape[0])
        # dobija se nasumicna permutacija indeksa kojom se mijesaju x i y trening skupovi
        x_train = x_train[permutation]
        y_train = y_train[permutation]
        # iterira se kroz taj izmjesani trening skup, od 0 do x_train velicine sa batch_size korakom 
        for i in range(0, x_train.shape[0], batch_size):
            model.zero_grad()
            x = x_train[i:i+batch_size]
            y = y_train[i:i+batch_size]
            # 'forward pass' sa isjecenim x podacima, kao gore za racunanje metrika
            y_pred = model.forward(x)
            # ali se ovde racuna greska izmedju predvidjenih y_pred i stvarnih y
            loss = loss_fn.forward(y_pred, y)
            # 'backward pass' za gradijent greske, racuna gradijente u odnosu na predvidjene izlaze
            grad = loss_fn.backward()
            # propagacija greske unazad
            model.backward(grad)
            # SGD optimajzer primjenjuje gradijente da se azuriraju parametri modela u smjeru koji smanjuje gresku
            optimizer.step()
            # broj pogresko klasifikovanih u trenutnom becu se racuna poredjenjem indeksa u maksimalnim vrijednostima predvidjanih i stvarnih vrijednosti i sumirajuci razlike
            total_misclassified += np.sum(np.argmax(y_pred, axis=1) != np.argmax(y, axis=1))
            # iskomentarisano zbog preglednijeg ispisa
            # if i % (batch_size * 1000) == 0:
            #     print(f'[Training] epoch={epoch} batch={i//batch_size:4}/{x_train.shape[0]//batch_size} loss={loss:.2f}')

        percent_misclassified = 100 * total_misclassified / x_train.shape[0]
        print(f'[Training] epoch={epoch} misclassified={percent_misclassified:3.2f}%')

        validation_loss = loss_fn.forward(model.forward(x_val), y_val)
        print(f"[Validation] epoch={epoch} loss={validation_loss:.2f}")
        if validation_loss < lowest_loss:
            lowest_loss = validation_loss
            num_bad_epochs = 0
        else:
            num_bad_epochs += 1
            if num_bad_epochs > tolerancy:
                print(f"Early stopping at epoch {epoch}")
                break

    total_misclassified = 0
    batch_size = 1000
    for i in range(0, x_test.shape[0], batch_size):
        x = x_test[i:i+batch_size]
        y = y_test[i:i+batch_size]
        y_pred = model.forward(x)
        total_misclassified += np.sum(np.argmax(y_pred, axis=1) != np.argmax(y, axis=1))
    percent_misclassified = 100 * total_misclassified / x_test.shape[0]
    print(f'[Test] misclassified={percent_misclassified:3.2f}%')

    val_prec, val_rec = evaluate_model(model, x_val, y_val)
    if val_prec > best_precision:
        best_precision = val_prec
        best_parameters = parameters

    eval_prec, eval_rec = evaluate_model(model, x_test, y_test)
    print(f"Precision after training: {eval_prec:.5f}")
    print(f"Recall after training: {eval_rec:.5f}")

print("Best hyperparameters: ", best_parameters)
print("Best precision: ", best_precision)

Parameters:  {'batch_size': 12, 'hidden_units': (100, 50), 'learning_rate': 0.35}
Precision before training: 0.00027
Recall before training: 0.01634
[Training] epoch=0 misclassified=33.20%
[Validation] epoch=0 loss=0.69
[Training] epoch=1 misclassified=29.31%
[Validation] epoch=1 loss=0.68
[Training] epoch=2 misclassified=27.98%
[Validation] epoch=2 loss=0.61
[Training] epoch=3 misclassified=27.01%
[Validation] epoch=3 loss=0.62
[Training] epoch=4 misclassified=26.26%
[Validation] epoch=4 loss=0.64
[Test] misclassified=28.79%
Precision after training: 0.71998
Recall after training: 0.71209
Parameters:  {'batch_size': 12, 'hidden_units': (100, 50), 'learning_rate': 0.035}
Precision before training: 0.22977
Recall before training: 0.40962
[Training] epoch=0 misclassified=35.26%
[Validation] epoch=0 loss=0.74
[Training] epoch=1 misclassified=31.25%
[Validation] epoch=1 loss=0.75
[Training] epoch=2 misclassified=30.29%
[Validation] epoch=2 loss=0.83
[Training] epoch=3 misclassified=29.73%


KeyboardInterrupt: ignored